In [1]:
from newsgac.pipelines.models import Pipeline
from newsgac import genres
from sklearn.pipeline import Pipeline as SKPipeline
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pprint


In [8]:
p = Pipeline.objects.get({'display_title': '20190228 SVC no stop words, lowercased'})
p.data_source.display_title

u'20190228 unbalanced train size=3443'

In [2]:
raw_texts = [article.raw_text for article in p.data_source.articles]
print(raw_texts[15])

Shaws The Philanderer : volmaakt , tijdloos toneel Voorstelling : The Philanderer van Bernard Shaw door Maatschappij Discordia . Toneelbeeld : Jan Joris Lamers ; regie : Frans Strijards ; spelers : Titus Muizelaar , Annet Kouwenhoven , Matthias de Koning , Jan Joris Lamers e.a . Gezien 27/2 Shaffy Theater , Amsterdam . Te zien t/m 2/3 aldaar ; daarna elders in het land . Door KESTER FRERIKS Als bezwaar tegen het toneelwerk van Bernard Shaw geldt wel dat hij geen drama 's creëerde waarin mensen van vlees en bloed optreden . Hij zou , zo luidt het oordeel , meer sociaal preker en pamflettist zijn dan auteur van onderhoudende toneelstukken met een voldoende aantal speelscènes , komische voorvallen , dramatische situaties en verhaallijnen . Het toneel dat Shaw schreef wordt ironisch betiteld als „ thedrama-of-nothing-but-ideas '' . Zijn personages lijken , in de slechtste betekenis van het woord , niets anders dan spreekbuis van de schrijver te zijn ; ze onderhouden elkaar op het toneel me

In [3]:
skp = p.sk_pipeline.get()
split_step = -1
pre_processing = SKPipeline(skp.steps[0:split_step])
classifier_pipeline = SKPipeline(skp.steps[split_step:])
predict_proba = classifier_pipeline.predict_proba
print( [s[0] for s in pre_processing.steps] )
print( [s[0] for s in classifier_pipeline.steps] )

['CleanOCR', 'Lowercase', 'StopWordRemoval', 'FeatureExtraction', 'RobustScaler']
['Classifier']


In [4]:
features = pre_processing.transform([raw_texts[15]]).toarray()[0]
feature_names_full = pre_processing.steps[-2][1].get_feature_names()
feature_names_short = [f.split('__')[-1] for f in feature_names_full]
feature_values = zip(feature_names_short, features)
best_50 = sorted(feature_values, key=lambda x: -x[1])[:50]
best_50_indices = [feature_names_short.index(f[0]) for f in best_50]
best_50_names = [f[0] for f in best_50]
best_50_dict = {f[0]: f[1] for f in best_50}

In [5]:
def features_to_proba(features, predict_proba):
    return sorted(zip(genres.genre_labels, predict_proba([features])[0]), key=lambda x: -x[1])
#features_to_proba(features, predict_proba)

In [6]:
def update_feature(**kwargs):
    for feature_name, feature_value in kwargs.items():
        features[best_50_indices[best_50_names.index(feature_name)]] = feature_value
    pprint.pprint(features_to_proba(features, predict_proba))

In [7]:
sliders = { feature_name: widgets.FloatSlider(
    min=0,
    max=2,
    step=0.01,
    value=best_50_dict[feature_name]
) for feature_name in best_50_names }

boxes = [ 
    widgets.HBox([
        widgets.Label(value=feature_name),
        sliders[feature_name]
    ], layout = { 'display': 'flex', 'justify_content': 'flex-end'}) for feature_name in best_50_names 
]

final_widget = widgets.VBox(boxes)

out = widgets.interactive_output(update_feature, sliders)

display(final_widget, out)

VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oTGFiZWwodmFsdWU9dSdodW4nKSwgRmxvYXRTbGlkZXIodmFsdWU9Mi4wLCBtYXg9Mi4wLCBzdGVwPTAuMDEpKSwgbGF5b3V0PUxheW91dCjigKY=


Output()